## 09: Вспомогательные функции — загрузка, визуализация зависимости и Спирмен

Файл содержит:
- загрузку данных из БД;
- функцию визуализации зависимости признака от целевой `corrosion_rate`;
- функцию расчёта Спирмена для списка признаков.


In [1]:
# Импорты и загрузка данных
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

if '../src' not in sys.path:
    sys.path.append('../src')
from database import load_corrosion_data_new as load_data
from analysis import CorrosionAnalyzer


In [2]:
# Настройки
sns.set(style='whitegrid')
TARGET = 'corrosion_rate_mm_per_year'

# Загрузка
DF = load_data()
assert TARGET in DF.columns, 'В данных отсутствует corrosion_rate_mm_per_year'

print(f"Данные: {len(DF):,} строк, {len(DF.columns)} колонок")
print('Колонки:')
print(sorted(DF.columns.tolist()))

Данные: 442,052 строк, 40 колонок
Колонки:
['ammonia_content', 'chlorine_content', 'co2_content', 'component', 'component_type_id', 'component_type_name', 'contour', 'corrosion_aggressiveness_index', 'corrosion_inhibitor_content', 'corrosion_rate_mm_per_year', 'cross_sectional_area', 'diameter_to_thickness_ratio', 'equipment', 'equipment_age_years', 'h2s_content', 'hydrochloric_acid_content', 'id', 'inner_diameter', 'installation', 'is_replaced', 'material_code', 'material_resistance_score', 'material_type', 'measurement', 'measurement_date', 'mms', 'nominal_thickness_mmc', 'operating_pressure', 'operating_temperature', 'outer_diameter', 'oxygen_content', 'radius', 'start_date_of_operation', 'tmin_mmc', 'total_acids', 'total_chlorine_compounds', 'total_sulfur_compounds', 'wall_loss_percentage', 'wall_thickness', 'water_content']


In [3]:
# Инициализация анализатора
analyzer = CorrosionAnalyzer(DF, target=TARGET)

# Определяем все доступные признаки для анализа
all_features = [
    # Химические параметры
    'water_content', 'h2s_content', 'chlorine_content', 'co2_content', 
    'oxygen_content', 'hydrochloric_acid_content', 'ammonia_content',
    'corrosion_inhibitor_content', 'total_sulfur_compounds', 
    'total_chlorine_compounds', 'total_acids', 'corrosion_aggressiveness_index',
    
    # Технологические параметры
    'operating_temperature', 'operating_pressure', 'material_resistance_score',
    'equipment_age_years', 'wall_thickness', 'radius', 'outer_diameter', 
    'inner_diameter', 'diameter_to_thickness_ratio', 'cross_sectional_area',
    
    # Геометрические параметры
    'nominal_thickness_mmc', 'tmin_mmc'
]

# Фильтруем только существующие признаки
available_features = [f for f in all_features if f in DF.columns]
print(f"📊 Доступно признаков для анализа: {len(available_features)}")

# Анализ заполненности данных
print(f"\n🔍 АНАЛИЗ ЗАПОЛНЕННОСТИ ДАННЫХ:")
print(f"   Всего наблюдений: {len(DF):,}")
print(f"   Заполненность целевой переменной: {DF[TARGET].notna().sum():,} ({DF[TARGET].notna().mean()*100:.1f}%)")
print(f"   Ненулевых значений коррозии: {(DF[TARGET] > 0).sum():,} ({(DF[TARGET] > 0).mean()*100:.1f}%)")

# ЭКСПЕРИМЕНТ 1: Корреляционный анализ
print("\n" + "="*80)
print("ЭКСПЕРИМЕНТ 1: КОРРЕЛЯЦИОННЫЙ АНАЛИЗ")
print("="*80)

# Вычисляем корреляции
corr_df, best_features = analyzer.compute_target_correlations(
    feature_cols=available_features,
    top_k=20,
    method='spearman',
    return_best_features=True
)

# Визуализация матрицы корреляций для топ-10 признаков
if len(best_features) >= 5:
    print(f"\n📈 ВИЗУАЛИЗАЦИЯ МАТРИЦЫ КОРРЕЛЯЦИЙ...")
    plot_correlation_matrix(DF, best_features[:10], TARGET)

# ЭКСПЕРИМЕНТ 2: Эксперименты с машинным обучением
print("\n" + "="*80)
print("ЭКСПЕРИМЕНТ 2: ЭКСПЕРИМЕНТЫ С МАШИННЫМ ОБУЧЕНИЕМ")
print("="*80)

# Определяем наборы признаков для экспериментов
feature_sets = {
    # Базовые эксперименты
    '01_top_5_correlated': best_features[:5],
    '01_top_8_correlated': best_features[:8],
    
    # Тематические эксперименты
    '02_chemical_focus': [f for f in best_features if any(chem in f for chem in ['content', 'acid', 'corrosion'])][:5],
    '03_process_focus': [f for f in best_features if any(proc in f for proc in ['temperature', 'pressure', 'resistance', 'age', 'thickness'])][:5],
    
    # Комбинированные эксперименты
    '04_mixed_best': best_features[:3] + 
                     [f for f in available_features if 'temperature' in f][:1] +
                     [f for f in available_features if 'pressure' in f][:1],
    
    # Полный набор (если данных достаточно)
    '05_all_available': best_features[:10],
}

# Запускаем эксперименты
all_results = []

for exp_name, features in feature_sets.items():
    if len(features) < 2:
        print(f"⚠️  Пропускаем {exp_name}: недостаточно признаков ({len(features)})")
        continue
    
    print(f"\n🚀 ЗАПУСК ЭКСПЕРИМЕНТА: {exp_name}")
    print(f"   Признаки: {features}")
    
    try:
        results = analyzer.run_experiment(
            columns=features,
            experiment_name=exp_name,
            test_size=0.2,
            random_state=42
        )
        all_results.append(results)
    except Exception as e:
        print(f"❌ Ошибка в эксперименте {exp_name}: {e}")

# Анализ и сравнение результатов
if all_results:
    print("\n" + "="*80)
    print("ФИНАЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ")
    print("="*80)
    
    # Объединяем все результаты
    combined_results = pd.concat(all_results, ignore_index=True)
    
    # Лучшие модели по R²
    print("\n🏆 ЛУЧШИЕ МОДЕЛИ ПО КАЧЕСТВУ (R²):")
    best_models = combined_results.sort_values('R2', ascending=False).head(10)
    for i, (_, row) in enumerate(best_models.iterrows(), 1):
        print(f"{i:2d}. {row['experiment']:25} | {row['model']:12} | "
              f"R² = {row['R2']:7.4f} | MAE = {row['MAE']:7.4f} | "
              f"признаков: {row['n_features']:2d}")
    
    # Визуализация сравнения экспериментов
    rf_results = combined_results[combined_results['model'] == 'RandomForest']
    
    if len(rf_results) > 1:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # R²
        axes[0, 0].bar(rf_results['experiment'], rf_results['R2'], color='lightblue', alpha=0.7)
        axes[0, 0].set_title('R² (больше → лучше)', fontweight='bold')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].grid(True, alpha=0.3)
        
        # MAE
        axes[0, 1].bar(rf_results['experiment'], rf_results['MAE'], color='lightcoral', alpha=0.7)
        axes[0, 1].set_title('MAE (меньше → лучше)', fontweight='bold')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].grid(True, alpha=0.3)
        
        # RMSE
        axes[1, 0].bar(rf_results['experiment'], rf_results['RMSE'], color='lightgreen', alpha=0.7)
        axes[1, 0].set_title('RMSE (меньше → лучше)', fontweight='bold')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
        
        # Количество признаков
        axes[1, 1].bar(rf_results['experiment'], rf_results['n_features'], color='lightyellow', alpha=0.7)
        axes[1, 1].set_title('Количество признаков', fontweight='bold')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.suptitle('Сравнение экспериментов Random Forest', fontsize=16, fontweight='bold', y=0.95)
        plt.tight_layout()
        plt.show()
    
    # Финальные выводы
    best_rf = rf_results.loc[rf_results['R2'].idxmax()]
    print(f"\n🎯 КЛЮЧЕВЫЕ ВЫВОДЫ:")
    print(f"   Лучший эксперимент: {best_rf['experiment']}")
    print(f"   Качество модели (R²): {best_rf['R2']:.4f}")
    print(f"   Точность (MAE): {best_rf['MAE']:.4f} мм/год")
    print(f"   Количество признаков: {best_rf['n_features']}")
    print(f"   Размер выборки: {best_rf['n_samples']:,}")

print("\n✅ АНАЛИЗ ЗАВЕРШЕН!")

📊 Доступно признаков для анализа: 25

🔍 АНАЛИЗ ЗАПОЛНЕННОСТИ ДАННЫХ:
   Всего наблюдений: 442,052
   Заполненность целевой переменной: 313,193 (70.8%)
   Ненулевых значений коррозии: 182,130 (41.2%)

ЭКСПЕРИМЕНТ 1: КОРРЕЛЯЦИОННЫЙ АНАЛИЗ
ТОП-20 ПРИЗНАКОВ ПО КОРРЕЛЯЦИИ С corrosion_rate_mm_per_year:
 1. *** wall_loss_percentage           : +0.9291 🡅
 2. *   nominal_thickness_mmc          : +0.1273 🡅
 3. *   wall_thickness                 : +0.1227 🡅
 4. *   cross_sectional_area           : +0.1208 🡅
 5. *   equipment_age_years            : +0.1146 🡅
 6. *   radius                         : +0.1112 🡅
 7. *   outer_diameter                 : +0.1112 🡅
 8. *   inner_diameter                 : +0.1040 🡅
 9.     tmin_mmc                       : +0.0901 🡅
10.     total_acids                    : -0.0893 🡇
11.     diameter_to_thickness_ratio    : +0.0854 🡅
12.     total_sulfur_compounds         : +0.0839 🡅
13.     h2s_content                    : +0.0688 🡅
14.     oxygen_content                 

NameError: name 'plot_correlation_matrix' is not defined